# Dialogue Management

## Two major components 

1. Understand what the user is saying

2. Decide what to say in response

## Language Understanding 

So far, we have been working with regular expressions.

Regular expressions are one technique we can use to program language understanding into a dialogue agent. 

We define what the agent understands by creating regular expressions.

The agent will understand what a user says if it matches the regular expression that is programmed.

If we take the phone type example from earlier:

In [2]:
import re
from typing import List
from re import Pattern

def regex_matcher(regex: Pattern, instr: str) -> List[str]:
    ts = [None] * regex.groups
    
    for t in regex.findall(instr):
        if isinstance(t, str): t = [t]
        for i, literal in enumerate(t):
            if ts[i] is None and literal:
                ts[i] = literal
    
    return ts

re_phone = re.compile(r'(?:\s|^)(apple|google|samsung)|(iphone|pixel|galaxy|android)(?:\s|,|\.|$)')

print(regex_matcher(re_phone, 'yes I have an iphone'))
print(regex_matcher(re_phone, 'yes I have google pixel'))
print(regex_matcher(re_phone, 'yes I have a galaxy phone'))

[None, 'iphone']
['google', 'pixel']
[None, 'galaxy']


We have programmed an agent that can understand when the user is sharing the type of phone that they have by recognizing the different companies and phone models.

Using the regular expression grouping capability, we can also specify what pieces of information we want the dialogue agent to extract if the user shares them.

For the phone type example, we are programming the dialogue agent to pick out the company name and the phone model name in such a way that this information can be stored and accessed later. 

It is important for dialogue agents to have this memory ability in order to make coherent responses to what the user says.

## Response Generation

Each utterance that a user says affects the dialogue situation that the agent is currently in. 

Depending on what the user says, we want the agent to be able to make different responses that make sense for the particular state of the dialogue.

If we take the starting turn of the phone conversation as an example:

In [4]:
from typing import Any
from types import SimpleNamespace

def turn_0(res: SimpleNamespace):
    s = 'S: are you using a smartphone?'
    u = input(s + '\nU: ')

    yn = regex_matcher(res.re_yn, u)
    phone = regex_matcher(res.re_phone, u)
    res.in_phone_company = phone[0]
    res.in_phone_name = phone[1]

    if any(phone):
        turn_1a(res)
    elif yn[0]:
        turn_1b(res)
    elif yn[1]:
        turn_1c(res)

    print('S: good bye!')

There are three different dialogue states that we can be in, depending on what the user says in response to our question.

1. The user has shared the particular type of phone that they have

2. The user has said yes to using a smartphone, but not specified a type

3. The user has said no to using a smartphone

In each of these cases, the agent should take a different course of action when responding to the user.

Since the amount of information that the agent knows about the user is different, its interactions should be tailored to these different situations in order to hold a fluent and natural conversation.

So far, we have been using conditional statements to make these dialogue decisions.

## Dialogue State Machine

There is name for this approach to dialogue: State Machine. 

State Machines are used to model many different processes, including dialogue. 

Anything that can be thought of as consisting of a sequence of actions resulting in different situations can be modelled as a State Machine.

There are two major components of a State Machine:

### 1. States

* Describe a specific point/situation in the process being defined 

### 2. Transitions

* Actions to move from one state to another

For dialogue, 

* transitions are the language being exchanged back and forth from either the agent or the user

* states are the different situations that result from specific sequences of language.

Below is a State Machine of the phone conversation we have been working with. 

* S marks states where the system is giving a response.

* U marks states where the user is giving a response. 

* The arrows show the response (or an approximation of it) that takes you from one state to another.

We will be using a python package that provides a straightforward interface for making state machines for dialogue.

Documentation and installation instructions can be found here: https://pypi.org/project/emora-stdm/

In this package, a state machine for dialogue is called a DialogueFlow.

Here is a simple example:

In [ ]:
from emora_stdm import DialogueFlow
from enum import Enum

# states are typically represented as an enum
class State(Enum):
    START = 0
    FAM_ANS = 1
    FAM_Y = 2
    FAM_N = 3
    FAM_ERR = 4
    WHATEV = 5

# initialize the DialogueFlow object, which uses a state-machine to manage dialogue
df = DialogueFlow(State.START)

# add transitions to create an arbitrary graph for the state machine
df.add_system_transition(State.START, State.FAM_ANS, '[!do you have a $F={brother, sister, son, daughter, cousin}]')
df.add_user_transition(State.FAM_ANS, State.FAM_Y, '[{yes, yea, yup, yep, i do, yeah}]')
df.add_user_transition(State.FAM_ANS, State.FAM_N, '[{no, nope}]')
df.add_system_transition(State.FAM_Y, State.WHATEV, 'thats great i wish i had a $F')
df.add_system_transition(State.FAM_N, State.WHATEV, 'ok then')
df.add_system_transition(State.FAM_ERR, State.WHATEV, 'im not sure i understand')

# each state that will be reached on the user turn should define an error transition if no other transition matches
df.set_error_successor(State.FAM_ANS, State.FAM_ERR)
df.set_error_successor(State.WHATEV, State.START)

df.run(debugging=False)

Since we have been focusing on regular expressions up until this point, the first thing we will discuss is how to specify the language expression for each transition between states in the package `emora-stdm`.

It uses a modified version of regular expressions - called `NatEx` - which simplifies much of the syntax, but ultimately the expression that you write is translated into a regular expression.

## NatEx

### Literal
```
'well hello'
```
directly match a literal substring

In [196]:
from emora_stdm import NatexNLU

natex_nlu = NatexNLU('well hello')

In [197]:
print(natex_nlu.match('well hello', debugging=False))

<regex.Match object; span=(0, 10), match='well hello'>


In [198]:
print(natex_nlu.match('well hi', debugging=False))

None


In [199]:
print(natex_nlu.match('well hello there', debugging=False))

None


### Disjunction
```
'{hi, hello}'
```
matches a substring containing exactly one term inside `{}`. In this case,
"hi" and "hello" both match.

In [200]:
natex_nlu = NatexNLU('{hi, hello}')

In [201]:
print(natex_nlu.match('hi', debugging=False))

<regex.Match object; span=(0, 2), match='hi'>


In [202]:
print(natex_nlu.match('hello', debugging=False))

<regex.Match object; span=(0, 5), match='hello'>


In [203]:
print(natex_nlu.match('hi hello', debugging=False))

None


In [204]:
print(natex_nlu.match('hey', debugging=False))

None


In [205]:
print(natex_nlu.match('hi everyone', debugging=False))

None


### Conjunction
```
'<everyone, hi>'
```
matches a substring that contains at least all terms inside `<>`. In this case, "hi everyone" and "oh well hi there everyone" both would match, but not
"hi".

In [206]:
natex_nlu = NatexNLU('<everyone, hi>')

In [207]:
print(natex_nlu.match('hi everyone', debugging=False))

<regex.Match object; span=(0, 11), match='hi everyone'>


In [208]:
print(natex_nlu.match('oh well hi there everyone', debugging=False))

<regex.Match object; span=(0, 25), match='oh well hi there everyone'>


In [209]:
print(natex_nlu.match('hi', debugging=False))

None


In [210]:
print(natex_nlu.match('everyone', debugging=False))

None


### Flexible sequence
```
'[well hello]'
```
matches as long as the utterance contains all terms inside `[]`,
and the terms are ordered properly within the utterance. Note that this expression matches any amount of characters
before and after the requisite sequence.

In [211]:
natex_nlu = NatexNLU('[well hello]')

In [212]:
print(natex_nlu.match('well hello', debugging=False))

<regex.Match object; span=(0, 10), match='well hello'>


In [213]:
print(natex_nlu.match('well hello there', debugging=False))

<regex.Match object; span=(0, 16), match='well hello there'>


In [214]:
print(natex_nlu.match('and then he said well hello there', debugging=False))

<regex.Match object; span=(0, 33), match='and then he said well hello there'>


In [215]:
print(natex_nlu.match('well hi there', debugging=False))

None


#### Slightly more complicated example:

It is important to point out that commas `,` are used to separate terms in our expressions. If there are multiple words separated by spaces contained within a single comma, then that entire string (including the spaces) will need to match. 

In [216]:
natex_nlu = NatexNLU('[he said, bob]')

In [217]:
print(natex_nlu.match('he said bob', debugging=False))

<regex.Match object; span=(0, 11), match='he said bob'>


In [218]:
print(natex_nlu.match('and the next that he said was bob', debugging=False))

<regex.Match object; span=(0, 33), match='and the next that he said was bob'>


In [219]:
print(natex_nlu.match('and bob was the next that he said', debugging=False))

None


### Inflexible sequence
```
'[!he, said, bob]'
```
matches an exact sequence of terms with no words inserted between
terms. The only utterance matching the example is "he said bob".

In [220]:
natex_nlu = NatexNLU('[!he, said, bob]')

In [221]:
print(natex_nlu.match('he said bob', debugging=False))

<regex.Match object; span=(0, 11), match='he said bob'>


In [222]:
print(natex_nlu.match('and the next thing is he said bob', debugging=False))

None


### Negation
```
[!i am -bad]
```
prepend `-` to negate the next term in the expression. The example
will match any expression starting with "i am" where "bad" does NOT
follow. Note that the scope of the negation extends to the end
of the substring due to limitations in regex.

In [223]:
natex_nlu = NatexNLU('[!i, am, -bad, at, math]')

In [224]:
print(natex_nlu.match('i am bad at math', debugging=False))

None


In [225]:
print(natex_nlu.match('i am good at math', debugging=False))

<regex.Match object; span=(0, 17), match='i am good at math'>


In [226]:
print(natex_nlu.match('i am good boy at math', debugging=False))

<regex.Match object; span=(0, 21), match='i am good boy at math'>


In [227]:
print(natex_nlu.match('i am bad', debugging=False))

None


### Regular expression
```
'/[A-Z a-z]+/'
```
substrings within `//` define a python regex directly.

If it is easier for some cases to use an actual regular expression, you can.

In [78]:
natex_nlu_natex = NatexNLU('{hi, hello}')

In [79]:
print(natex_nlu_natex.match('hi', debugging=False))

<regex.Match object; span=(0, 2), match='hi'>


In [80]:
print(natex_nlu_natex.match('hello', debugging=False))

<regex.Match object; span=(0, 5), match='hello'>


In [81]:
print(natex_nlu_natex.match('bye', debugging=False))

None


In [82]:
natex_nlu_regex = NatexNLU('/hi|hello/')

In [83]:
print(natex_nlu_regex.match('hi', debugging=False))

<regex.Match object; span=(0, 2), match='hi'>


In [84]:
print(natex_nlu_regex.match('hello', debugging=False))

<regex.Match object; span=(0, 5), match='hello'>


In [85]:
print(natex_nlu_regex.match('bye', debugging=False))

None


### Nesting
```
'[!{hi, hello} [how, weekend]]'
```
would match "hi how was your weekend", "oh hello so how is the
weekend going", ...

This example has:
* Inflexible Sequence

with nested

* Disjunction
* Flexible Sequence

In [228]:
natex_nlu = NatexNLU('[!{hi, hello} [how, weekend]]')

In [229]:
print(natex_nlu.match('hello how was your weekend', debugging=False))

<regex.Match object; span=(0, 26), match='hello how was your weekend'>


Matches because:
* Meets the requirements of the Inflexible Sequence
* Meets the requirements of the Disjunction
* Meets the requirements of the Flexible Sequence

In [230]:
print(natex_nlu.match('oh hi how was your weekend', debugging=False))

None


Doesn't match because of Inflexible Sequence
* there cannot be any leading words before the disjunction

In [231]:
print(natex_nlu.match('hi there how was your weekend', debugging=False))

<regex.Match object; span=(0, 29), match='hi there how was your weekend'>


Matches because of inner Flexible Sequence
* allows for extra words around it (even though it is nested inside of Inflexible Sequence)

### Variable assignment
```
'[!i am $f={good, bad}]'
```
using `$var=` will assign variable `var` to the next term in
the expression. The variable will persist until overwritten,
and can be referenced in future NLU or NLG expressions.
The example would match either "i am good" or "i am bad", and
assigns variable "f" to either "good" or "bad" depending
on what the user said.

In [232]:
vars_dict = {}
natex_nlu = NatexNLU('[!i am $f={good, bad}]')

In [233]:
print(natex_nlu.match('i am good', debugging=False, vars=vars_dict))

<regex.Match object; span=(0, 9), match='i am good'>


In [234]:
vars_dict

{'f': 'good'}

In [235]:
print(natex_nlu.match('i am bad', debugging=False, vars=vars_dict))

<regex.Match object; span=(0, 8), match='i am bad'>


In [236]:
vars_dict

{'f': 'bad'}

In [237]:
vars_dict = {}
natex_nlu = NatexNLU('[!i am $f=[the, best]]')

In [238]:
print(natex_nlu.match('i am the very best artist', debugging=False, vars=vars_dict))

<regex.Match object; span=(0, 25), match='i am the very best artist'>


In [239]:
vars_dict

{'f': 'the very best artist'}

### Variable reference
```
[!why are you $f today]
```
using `$` references a previously assigned variable. If no such
variable exists, the expression as a whole returns with no match.
The example would match "why are you good today" if `f="good"`, 
but would not match if `f="bad"`

In [240]:
vars_dict = {"f": "good"}
natex_nlu = NatexNLU('[!why are you $f today]')

In [241]:
print(natex_nlu.match('why are you good today', debugging=False, vars=vars_dict))

<regex.Match object; span=(0, 22), match='why are you good today'>


In [242]:
print(natex_nlu.match('why are you bad today', debugging=False, vars=vars_dict))

None


In [243]:
vars_dict = {}
natex_nlu_statement = NatexNLU('[!i am $f={good, bad}]')
print(natex_nlu_statement.match('i am bad', debugging=False, vars=vars_dict))
print(vars_dict)

<regex.Match object; span=(0, 8), match='i am bad'>
{'f': 'bad'}


In [244]:
print(natex_nlu.match('why are you good today', debugging=False, vars=vars_dict))

None


In [245]:
print(natex_nlu.match('why are you bad today', debugging=False, vars=vars_dict))

<regex.Match object; span=(0, 21), match='why are you bad today'>


## Phone Conversation: Regex to NatEx

Let's return to the phone conversation we have been working with.

### Response: Yes

In [11]:
from emora_stdm import NatexNLU

REGEX = "(?:\s|^)(yes|yeah)(?:\s|,|\.|$)"

yes = r"[{yes,yeah}]"
yes_natex = NatexNLU(yes)
print(yes_natex.match("yes i have an iphone"))
print()

<regex.Match object; span=(0, 20), match='yes i have an iphone'>



### Response: No

In [247]:
REGEX = "(?:\s|^)(no|not really)(?:\s|,|\.|$)(?:.*)"

no = r"[{no,not really}]"
no_natex = NatexNLU(no)
print(no_natex.match("actually not really"))
print()

<regex.Match object; span=(0, 19), match='actually not really'>



### Response: Phone Model

In [18]:
REGEX = "(?:\s|^)(apple|google|samsung)|(iphone|pixel|galaxy|android)(?:\s|,|\.|$)"

pcm_vars = {}
phone_company_model = r"{[$company={apple,google,samsung},$model={iphone,pixel,galaxy,android}],[$company={apple,google,samsung}],[$model={iphone,pixel,galaxy,android}]}"
phone_company_model_natex = NatexNLU(phone_company_model)
print(phone_company_model_natex.match("the samsung android one", vars=pcm_vars))
print(pcm_vars)
print()

<regex.Match object; span=(0, 23), match='the samsung android one'>
{'company': 'samsung', 'model': 'android'}



### Response: Duration

In [249]:
REGEX = "(?:\s|^)(\d+)(?:\s|-)+(month|year)"

t_vars = {}
timeframe = r"[$number=/\d+/,$time_type={month,months,year,years}]"
timeframe_natex = NatexNLU(timeframe)
print(timeframe_natex.match("6 months", vars=t_vars))
print(t_vars)
print()

<regex.Match object; span=(0, 8), match='6 months'>
{'number': '6', 'time_type': 'months'}



### Response: From Date

In [250]:
REGEX = "(?:\s|^)(?:since|from)\s(?:(january|february|march|april|may|june|july|august|september|october|november|december)\s)?(\d{2,4})"

d_vars = {}
date = r"[!{since,from},$month={january,february,march,april,may,june,july,august,september,october,november,december},$year=/\d{2,4}/]"
date_natex = NatexNLU(date)
print(date_natex.match("since february 2019", vars=d_vars))
print(d_vars)
print()

<regex.Match object; span=(0, 19), match='since february 2019'>
{'month': 'february', 'year': '2019'}



### Response: Version

In [121]:
REGEX = "(?:\s|^)(?:iphone|version)\s(\d+s?(?: (?:plus|max))?)(?:\s|,|\.|$)"

im_vars = {}
iphone_model = r"[$iphone_model=[!{iphone,version},{/\d+/,/\d+s/},{plus,max}]]"
iphone_model_natex = NatexNLU(iphone_model)
print(iphone_model_natex.match("iphone 10s plus", vars=im_vars))
print(im_vars)
print()

<regex.Match object; span=(0, 15), match='iphone 10s plus'>
{'iphone_model': 'iphone 10s plus'}



### Macro: Predict Phone Version

In [1]:
from emora_stdm import Macro

class VERSION(Macro):
    
    def run(self, ngrams, vars, args):
        
        d_iphone = {
            2019: [(9, ['11', '11 pro', '11 pro max'])], 
            2018: [(9, ['10s', '10s max'])], 
            2017: [(11, ['10']), (9, ['8', '8 plus'])], 
            2016: [(9, ['7', '7 plus'])], 
            2015: [(9, ['6s', '6s plus'])], 
            2014: [(9, ['6', '6 plus'])]}
        
        d_month_to_number = {
            month: i for i, month in enumerate(
                ['january','february','march','april','may','june',
                 'july','august','september','october','november','december'], start=1)}

        year = None
        curr_year, curr_month = 2020, 1
        
        if 'number' in vars and 'time_type' in vars:
            d = int(vars['number'])
            m = vars['time_type']
            if m.startswith('year'):
                year = curr_year - d
                month = curr_month
            elif m.startswith('month'):
                year = curr_year - int(d / 12)
                month = curr_month - (d % 12)
                if month <= 0:
                    month_diff = abs(month)
                    month = 12 - month_diff
                    year -= 1
                    
        elif 'year' in vars:    
            year = int(vars['year'])
            if year <= 20: year += 2000
            if 'month' in vars:
                month = d_month_to_number[vars['month']]
            else:
                month = 1

        r = d_iphone.get(year, None)
        if r:
            v = next((s_models for s_month, s_models in r if s_month >= month), [])
            return ' or '.join(v)


## Phone Conversation State Machine

![dialogue_management_cs329_fig%20%284%29.svg](attachment:res/dialogue_management_cs329_fig%20%284%29.svg)

In [ ]:
from emora_stdm import DialogueFlow
from enum import Enum

class State(Enum):
    S0 = 0
    U0 = 1
    S1A = 2
    U1A = 3
    S2A = 4
    S2B = 5
    U2A = 6
    U2B = 7
    S3 = 8
    END = 9
    ERR = 10

df = DialogueFlow(State.S0, initial_speaker=DialogueFlow.Speaker.SYSTEM, macros={"VERSION":VERSION()})

# FIRST TURN

df.add_system_transition(State.S0, State.U0, r'[!are you using a smartphone"?"]')

# SECOND TURN

yesphone_natex = r"[{yes,yeah}, {[$company={apple,google,samsung},$model={iphone,pixel,galaxy,android}],[$company={apple,google,samsung}],[$model={iphone,pixel,galaxy,android}]}]"
df.add_user_transition(State.U0, State.S1A, yesphone_natex)

df.add_system_transition(State.S1A, State.U1A, r'[!how long have you been using $model"?"]')

# THIRD TURN

time_natex = r"{[$number=/\d+/,$time_type={month,months,year,years}], [!{since,from},$month={january,february,march,april,may,june,july,august,september,october,november,december},$year=/\d{2,4}/]}"
df.add_user_transition(State.U1A, State.S2A, time_natex)

df.add_system_transition(State.S2A, State.U2A, r'[!oh are you using $model #VERSION"?"]')

df.set_error_successor(State.U1A, error_successor=State.S2B) # if we don't get any time information, take the error transition to S2B

df.add_system_transition(State.S2B, State.U2B, r'[! what version of phone are you using"?"]')

# FINAL TURN

df.add_user_transition(State.U2A, State.END, "/.*/")
df.add_user_transition(State.U2B, State.END, "/.*/")

# DOES NOT HANDLE S3 CURRENTLY
# TO DO FOR QUIZ 2

df.add_system_transition(State.END, State.END, "END REACHED")

# ERROR STATES
df.add_system_transition(State.ERR, State.ERR, r"[!i have failed]")
df.set_error_successor(State.U0, error_successor=State.ERR)
df.set_error_successor(State.U2A, error_successor=State.ERR)
df.set_error_successor(State.U2B, error_successor=State.ERR)
df.set_error_successor(State.ERR, error_successor=State.ERR)
df.set_error_successor(State.END, error_successor=State.END)


df.run(debugging=False)

## Using an Ontology

*Ontology:* Hierarchy of words based on categorical groupings 

Instead of specifying a disjunction of words in NatEx, it is often easier and more flexible to reference an ontology group.

You can make any arbitrary ontology that you want by building a python dictionary of the form:
```  
{
    "ontology": {
        "category1": [
            "item1",
            "item2",
            ...
        ],
        "category2": [
            ...
        ]
    }
    
}
```

ONT() is a built-in macro for referencing an ontology group in NatEx.

In [ ]:
from emora_stdm import KnowledgeBase, DialogueFlow

class State(Enum):
    START = 0
    MAMMAL = 1
    BIRD = 2
    ERR = 3
    
animal_d = {"ontology": {"ontmammal": ["cat", "dog"], "ontbird": ["parrot","dove","crow"]}}

knowledge = KnowledgeBase()
knowledge.load_json(animal_d)
df = DialogueFlow(State.START, initial_speaker=DialogueFlow.Speaker.USER, kb=knowledge)
df.add_state(State.START, error_successor=State.ERR)
df.add_state(State.MAMMAL, error_successor=State.ERR)
df.add_state(State.BIRD, error_successor=State.ERR)
df.add_state(State.ERR, error_successor=State.ERR)

df.add_user_transition(State.START, State.MAMMAL, "[#ONT(ontmammal)]")
df.add_user_transition(State.START, State.BIRD, "[#ONT(ontbird)]")
df.add_system_transition(State.MAMMAL, State.START, '[!thats a mammal"," enter another animal]')
df.add_system_transition(State.BIRD, State.START, '[!thats a bird"," enter another animal]')
df.add_system_transition(State.ERR, State.START, '[!i dont know that one"," enter another animal]')

df.run(debugging=False)

U: cat
S: thats a mammal , enter another animal
U: i like dog
S: thats a mammal , enter another animal
U: do you like parrot
S: thats a bird , enter another animal
